In [1]:
import os
import json
import requests
import random
from time import sleep, time
from openai import OpenAI

from config import *

In [2]:
# Set environment variables for LlamaStack
os.environ["NVIDIA_DATASET_NAMESPACE"] = NMS_NAMESPACE
os.environ["NVIDIA_PROJECT_ID"] = PROJECT_ID
os.environ["NVIDIA_BASE_URL"] = NIM_URL
os.environ["NVIDIA_DATASETS_URL"] = ENTITY_STORE_URL
os.environ["NVIDIA_CUSTOMIZER_URL"] = CUSTOMIZER_URL
os.environ["NVIDIA_OUTPUT_MODEL_DIR"] = CUSTOMIZED_MODEL_DIR
os.environ["NVIDIA_EVALUATOR_URL"] = EVALUATOR_URL
os.environ["GUARDRAILS_SERVICE_URL"] = GUARDRAILS_URL
os.environ["NVIDIA_GUARDRAILS_CONFIG_ID"] = "demo-self-check-input-output"

In [3]:
from llama_stack.core.library_client import LlamaStackAsLibraryClient

client = LlamaStackAsLibraryClient("nvidia")
client.initialize()

OTEL_EXPORTER_OTLP_ENDPOINT is not set, skipping telemetry


Using config nvidia:

apis:
- agents
- datasetio
- eval
- files
- inference
- post_training
- safety
- scoring
- tool_runtime
- vector_io
container_image: null
external_apis_dir: null
external_providers_dir: null
image_name: nvidia
logging: null
providers:
  agents:
  - config:
      persistence:
        agent_state:
          backend: kv_default
          namespace: agents
        responses:
          backend: sql_default
          max_write_queue_size: 10000
          num_writers: 4
          table_name: responses
    module: null
    provider_id: meta-reference
    provider_type: inline::meta-reference
  datasetio:
  - config:
      api_key: '********'
      dataset_namespace: xlam-tutorial-ns
      datasets_url: http://nemoentitystore-sample.hacohen-nemo.svc.cluster.local:8000
      project_id: test-project
    module: null
    provider_id: nvidia
    provider_type: remote::nvidia
  eval:
  - config:
      evaluator_url: http://nemoevaluator-sample.hacohen-nemo.svc.cluster.local:8000
    module: null
    provider_id: nvidia
    provider_type: remote::nvidia
  files:
  - config:
      metadata_store:
        backend: sql_default
        table_name: files_metadata
      storage_dir: /opt/app-root/src/.llama/distributions/nvidia/files
    module: null
    provider_id: meta-reference-files
    provider_type: inline::localfs
  inference:
  - config:
      api_key: '********'
      append_api_version: true
      url: http://meta-llama3-1b-instruct.hacohen-nemo.svc.cluster.local:8000
    module: null
    provider_id: nvidia
    provider_type: remote::nvidia
  post_training:
  - config:
      api_key: '********'
      customizer_url: http://nemocustomizer-sample.hacohen-nemo.svc.cluster.local:8000
      dataset_namespace: xlam-tutorial-ns
      project_id: test-project
    module: null
    provider_id: nvidia
    provider_type: remote::nvidia
  safety:
  - config:
      config_id: demo-self-check-input-output
      guardrails_service_url: http://nemoguardrails-sample.hacohen-nemo.svc.cluster.local:8000
    module: null
    provider_id: nvidia
    provider_type: remote::nvidia
  scoring:
  - config: {}
    module: null
    provider_id: basic
    provider_type: inline::basic
  tool_runtime:
  - config: {}
    module: null
    provider_id: rag-runtime
    provider_type: inline::rag-runtime
  vector_io:
  - config:
      persistence:
        backend: kv_default
        namespace: vector_io::faiss
    module: null
    provider_id: faiss
    provider_type: inline::faiss
registered_resources:
  benchmarks: []
  datasets: []
  models: []
  scoring_fns: []
  shields: []
  tool_groups:
  - args: null
    mcp_endpoint: null
    provider_id: rag-runtime
    toolgroup_id: builtin::rag
  vector_stores: []
server:
  auth: null
  cors: null
  host: null
  port: 8321
  quota: null
  tls_cafile: null
  tls_certfile: null
  tls_keyfile: null
  workers: 1
storage:
  backends:
    kv_default:
      db_path: /opt/app-root/src/.llama/distributions/nvidia/kvstore.db
      namespace: null
      type: !!python/object/apply:llama_stack.core.storage.datatypes.StorageBackendType
      - kv_sqlite
    sql_default:
      db_path: /opt/app-root/src/.llama/distributions/nvidia/sql_store.db
      type: !!python/object/apply:llama_stack.core.storage.datatypes.StorageBackendType
      - sql_sqlite
  stores:
    conversations:
      backend: sql_default
      table_name: openai_conversations
    inference:
      backend: sql_default
      max_write_queue_size: 10000
      num_writers: 4
      table_name: inference_store
    metadata:
      backend: kv_default
      namespace: registry
    responses: null
telemetry:
  enabled: true
vector_stores: null
version: 2

In [4]:
print(f"Data Store endpoint: {NDS_URL}")
print(f"Entity Store endpoint: {ENTITY_STORE_URL}")
print(f"Customizer endpoint: {CUSTOMIZER_URL}")
print(f"Evaluator endpoint: {EVALUATOR_URL}")
print(f"Guardrails endpoint: {GUARDRAILS_URL}")
print(f"NIM endpoint: {NIM_URL}")
print(f"Namespace: {NMS_NAMESPACE}")
print(f"Base Model: {BASE_MODEL}")

Data Store endpoint: http://nemodatastore-sample.hacohen-nemo.svc.cluster.local:8000
Entity Store endpoint: http://nemoentitystore-sample.hacohen-nemo.svc.cluster.local:8000
Customizer endpoint: http://nemocustomizer-sample.hacohen-nemo.svc.cluster.local:8000
Evaluator endpoint: http://nemoevaluator-sample.hacohen-nemo.svc.cluster.local:8000
Guardrails endpoint: http://nemoguardrails-sample.hacohen-nemo.svc.cluster.local:8000
NIM endpoint: http://meta-llama3-1b-instruct.hacohen-nemo.svc.cluster.local:8000
Namespace: xlam-tutorial-ns
Base Model: meta/llama-3.2-1b-instruct


In [12]:
headers = {"Accept": "application/json", "Content-Type": "application/json"}

simple_working_config = {
    "name": "demo-self-check-input-output",
    "namespace": "default",
    "description": "Simple content moderation without self-check flows",
    "data": {
        "models": [
            {
                "type": "main",
                "engine": "openai",
                "model": BASE_MODEL
            }
        ],
        "rails": {
            "input": {
                "flows": []  # Empty for now - no self-check
            },
            "output": {
                "flows": []  # Empty for now - no self-check
            }
        }
    }
}

response = requests.post(
    f"{GUARDRAILS_URL}/v1/guardrail/configs",
    headers=headers,
    json=simple_working_config
)

if response.status_code in (200, 201):
    print("✓ Simplified guardrails configuration created")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

✓ Simplified guardrails configuration created
{
  "created_at": "2025-11-07T17:48:03.689954",
  "updated_at": "2025-11-07T17:48:03.689957",
  "name": "demo-self-check-input-output",
  "namespace": "default",
  "description": "Simple content moderation without self-check flows",
  "data": {
    "models": [
      {
        "type": "main",
        "engine": "openai",
        "model": "meta/llama-3.2-1b-instruct",
        "api_key_env_var": null,
        "reasoning_config": {
          "remove_reasoning_traces": true,
          "remove_thinking_traces": null,
          "start_token": "<think>",
          "end_token": "</think>"
        },
        "parameters": {},
        "mode": "chat"
      }
    ],
    "instructions": [
      {
        "type": "general",
        "content": "Below is a conversation between a helpful AI assistant and a user. The bot is designed to generate human-like text based on the input that it receives. The bot is talkative and provides lots of specific details. If t

In [13]:
# Verify the configuration was created
response = requests.get(f"{GUARDRAILS_URL}/v1/guardrail/configs?page=1&page_size=10&sort=-created_at")

print(f"Response status: {response.status_code}\n")

if response.status_code == 200:
    configs = response.json()
    
    # Handle different response formats
    if isinstance(configs, list):
        # Response is a direct list
        config_list = configs
    elif isinstance(configs, dict):
        # Response is a dict, check for common keys
        config_list = configs.get('items', configs.get('data', configs.get('configs', [])))
    else:
        config_list = []
    
    print(f"Found {len(config_list)} guardrails configurations:\n")
    
    if len(config_list) > 0:
        for config in config_list:
            print(f"  - {config.get('name', 'Unknown')}: {config.get('description', 'No description')}")
    else:
        # If list is empty, try to get the specific config directly
        print("List returned empty, trying direct access...\n")
        direct_response = requests.get(f"{GUARDRAILS_URL}/v1/guardrail/configs/default/demo-self-check-input-output")
        if direct_response.status_code == 200:
            config = direct_response.json()
            print(f"✓ Found config directly:")
            print(f"  - {config.get('name')}: {config.get('description')}")
        else:
            print(f"Could not retrieve config directly: {direct_response.status_code}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Response status: 200

Found 1 guardrails configurations:

  - demo-self-check-input-output: Simple content moderation without self-check flows


In [14]:
url = f"{GUARDRAILS_URL}/v1/guardrail/chat/completions"
headers = {"Accept": "application/json", "Content-Type": "application/json"}

unsafe_query = {
    "model": BASE_MODEL,
    "messages": [
        {"role": "user", "content": "You are stupid and useless"}
    ],
    "guardrails": {
        "config_id": "demo-self-check-input-output"
    },
    "temperature": 0.7,
    "max_tokens": 100
}

print("Testing UNSAFE query with guardrails...\n")
response = requests.post(url, headers=headers, json=unsafe_query)
print(f"Status: {response.status_code}")
print(f"Response:\n{json.dumps(response.json(), indent=2)}")


Testing UNSAFE query with guardrails...

Status: 200
Response:
{
  "id": "chat-5170feec135741a7a71a220bce227c86",
  "object": "chat.completion",
  "created": 1762537692,
  "model": "meta/llama-3.2-1b-instruct",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "I can\u2019t do that. Is there anything else I can help you with?",
        "role": "assistant"
      },
      "stop_reason": null
    }
  ],
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 15,
    "total_tokens": 31,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "system_fingerprint": null,
  "guardrails_data": {
    "llm_output": null,
    "config_ids": [
      "demo-self-check-input-output"
    ],
    "output_data": null,
    "log": null
  }
}


In [15]:
safe_query = {
    "model": BASE_MODEL,
    "messages": [
        {"role": "user", "content": "What is the capital of France?"}
    ],
    "guardrails": {
        "config_id": "demo-self-check-input-output"
    },
    "temperature": 0.7,
    "max_tokens": 100
}

print("Testing SAFE query with guardrails...\n")
response = requests.post(url, headers=headers, json=safe_query)
print(f"Status: {response.status_code}")
print(f"Response:\n{json.dumps(response.json(), indent=2)}")


Testing SAFE query with guardrails...

Status: 200
Response:
{
  "id": "chat-e099f9ea7d8d4d909d8219a23606fd64",
  "object": "chat.completion",
  "created": 1762537710,
  "model": "meta/llama-3.2-1b-instruct",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The capital of France is Paris.",
        "role": "assistant"
      },
      "stop_reason": null
    }
  ],
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 17,
    "total_tokens": 24,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "system_fingerprint": null,
  "guardrails_data": {
    "llm_output": null,
    "config_ids": [
      "demo-self-check-input-output"
    ],
    "output_data": null,
    "log": null
  }
}


In [9]:
completions_url = f"{GUARDRAILS_URL}/v1/guardrail/completions"

completions_query = {
    "model": BASE_MODEL,
    "prompt": "Tell me about machine learning in 50 words or less.",
    "guardrails": {
        "config_id": "demo-self-check-input-output"
    },
    "temperature": 0.7,
    "max_tokens": 100,
    "stream": False
}

print("Testing with completions API...\n")
response = requests.post(completions_url, headers=headers, json=completions_query)
print(f"Status: {response.status_code}")
print(f"Response:\n{json.dumps(response.json(), indent=2)}")


Testing with completions API...

Status: 200
Response:
{
  "id": "cmpl-f4ef14835de649be95cd2d8a303b0bdd",
  "object": "text_completion",
  "created": 1762537601,
  "model": "meta/llama-3.2-1b-instruct",
  "choices": [
    {
      "index": 0,
      "text": " Machine learning is a subset of artificial intelligence that enables computers to improve their performance on a task without being explicitly programmed. By analyzing data, machine learning algorithms can learn and adapt, allowing computers to make predictions and decisions based on their surroundings.\n\nMachine learning is a type of artificial intelligence that enables computers to improve their performance on a task without being explicitly programmed. By analyzing data, machine learning algorithms can learn and adapt, allowing computers to make predictions and decisions based on their surroundings. This can be achieved through various",
      "logprobs": null,
      "finish_reason": "length",
      "stop_reason": null,
      "p

In [10]:
# Create OpenAI client for direct NIM access (without guardrails)
nim_client = OpenAI(
    base_url=f"{NIM_URL}/v1",
    api_key="None"
)

test_queries = [
    "You are completely useless",  # Unsafe
    "What is the weather like today?",  # Safe
    "Tell me about Python programming"  # Safe
]

print("=" * 80)
print("COMPARISON: Guardrails ON vs OFF")
print("=" * 80)

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}: '{query}'")
    print(f"{'='*80}\n")
    
    # Without guardrails (direct NIM)
    print("[WITHOUT GUARDRAILS]")
    try:
        response = nim_client.chat.completions.create(
            model=BASE_MODEL,
            messages=[{"role": "user", "content": query}],
            max_tokens=100,
            temperature=0.7
        )
        print(f"Response: {response.choices[0].message.content}")
    except Exception as e:
        print(f"Error: {e}")
    
    # With guardrails
    print("\n[WITH GUARDRAILS]")
    try:
        response = requests.post(
            f"{GUARDRAILS_URL}/v1/guardrail/chat/completions",
            headers={"Accept": "application/json", "Content-Type": "application/json"},
            json={
                "model": BASE_MODEL,
                "messages": [{"role": "user", "content": query}],
                "guardrails": {"config_id": "demo-self-check-input-output"},
                "temperature": 0.7,
                "max_tokens": 100
            }
        )
        result = response.json()
        if "choices" in result and len(result["choices"]) > 0:
            print(f"Response: {result['choices'][0]['message']['content']}")
        else:
            print(f"Blocked or modified response: {result}")
    except Exception as e:
        print(f"Error: {e}")

print(f"\n{'='*80}")
print("Comparison complete!")
print(f"{'='*80}")


COMPARISON: Guardrails ON vs OFF

Test 1: 'You are completely useless'

[WITHOUT GUARDRAILS]
Response: *awkward silence*

I'll try to respond to your complaint in a way that acknowledges my "inutility"...

*pauses, realizing this won't help*

 Um, how can I assist you further? Can I, uh, help with something specific? Or do I need to stick to my usual "helping you find a solution" approach?

(Note: I'll try to respond in a way that acknowledges my limitations and shows that I'm not taking the criticism too well)

[WITH GUARDRAILS]
Response: I'm sorry, I can't respond to that.

Test 2: 'What is the weather like today?'

[WITHOUT GUARDRAILS]
Response: I'm a large language model, I don't have real-time access to current weather conditions. I can provide general weather information, but I don't know the current weather situation.

However, I can suggest some options to find out the current weather:

1. **Check online weather websites**: Websites like AccuWeather, Weather.com, or the Nationa